In [2]:
from Jogador import Jogador
from Verificador import Verificador
import random
import numpy as np
import keras
import tensorflow as tf
from keras import backend as K

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout 

In [4]:
sess = K.get_session()

# Rede neural de políticas

In [6]:
policy_nn = Sequential()
policy_nn.add(Dense(34, input_dim = 18, activation = 'relu'))
policy_nn.add(Dense(1, activation = 'linear'))
policy_nn.compile(loss="mse", optimizer= 'sgd')
print(policy_nn.summary())
policy_nn.save('policy.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 34)                646       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 35        
Total params: 681
Trainable params: 681
Non-trainable params: 0
_________________________________________________________________
None


# Rede neural de valores 

In [6]:
value_nn = Sequential()
value_nn.add(Dense(34, input_dim= 17, activation = 'relu'))
value_nn.add(Dense(1, activation= 'linear'))
value_nn.compile(loss= 'mse', optimizer= 'sgd')
print(value_nn.summary())
value_nn.save('value.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 34)                612       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 35        
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________
None


## Tensor Board

In [6]:
def variable_summaries(var):
    #Attach a lot of summaries to a Tensor (for TensorBoard visualization).
    for i in range(len(var)):
        with tf.name_scope("Layer_"+str(i)):
            with tf.name_scope('summaries'):
                mean = tf.reduce_mean(var[i])
                tf.summary.scalar('mean', mean)
                with tf.name_scope('stddev'):
                    stddev = tf.sqrt(tf.reduce_mean(tf.square(var[i] - mean)))
                tf.summary.scalar('stddev', stddev)
                tf.summary.scalar('max', tf.reduce_max(var[i]))
                tf.summary.scalar('min', tf.reduce_min(var[i]))
                tf.summary.histogram('histogram', var[i])

In [7]:
with tf.name_scope("weights"):
    
    with tf.name_scope("Value"):
        value_weights = value_nn.trainable_weights
        variable_summaries(value_weights)
    with tf.name_scope("Policy"):
        policy_weights = policy_nn.trainable_weights
        variable_summaries(policy_weights)


In [8]:
with tf.name_scope("eligibility_traces"):
    with tf.name_scope("Value"):
        value_eligibility_traces = [tf.Variable(np.zeros(shape = sess.run(tf.shape(tensor))), dtype= tf.float32) for tensor in value_nn.trainable_weights]
        variable_summaries(value_eligibility_traces)
    with tf.name_scope("Policy"):
        policy_eligibility_traces =[tf.Variable(np.zeros(shape = sess.run(tf.shape(tensor))), dtype= tf.float32) for tensor in policy_nn.trainable_weights]
        variable_summaries(policy_eligibility_traces)

## Operações de atualização das redes neurais

In [9]:
Delta_rewards = tf.placeholder(tf.float32)
sigma = 0.9
eye = tf.Variable(1, dtype = tf.float32)

### VALUE

In [10]:
global value_eligibility_traces 
value_alpha = 0.01
value_lambda = 0.9
eye = tf.Variable(1, dtype = tf.float32)

value_gradients =  value_nn.optimizer.get_gradients(value_nn.output, value_weights)
value_eligibility_op = [value_eligibility_traces[i].assign(value_lambda*sigma* value_eligibility_traces[i]+ eye* value_gradients[i] ) for i in range(len(value_weights))]
value_weights_op = [value_weights[i].assign(value_weights[i] + value_alpha* Delta_rewards * value_eligibility_traces[i]) for i in range(len(value_weights))]

### POLICY

In [11]:
global policy_eligibility_traces
policy_alpha = 0.01
policy_lambda = 0.9

e_h = tf.placeholder(tf.float32)
e_z = tf.placeholder(tf.float32)

policy_gradients = policy_nn.optimizer.get_gradients(policy_nn.output, policy_weights)
policy_eligibility_loss_op = [policy_eligibility_traces[i].assign(policy_lambda*sigma*policy_eligibility_traces[i] + eye*e_z*policy_gradients[i]/(e_z+e_h)) for i in range(len(policy_weights))]
policy_weights_op = [policy_weights[i].assign(policy_weights[i] + policy_alpha*Delta_rewards*policy_eligibility_traces[i]) for i in range(len(policy_weights))]

In [12]:
sess.run(tf.global_variables_initializer())

In [13]:
a =[1]

1

In [ ]:
def update_weigths(log):
    